In [35]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [36]:
def get_jobs(job_name, num_jobs, debugging):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/leo/Desktop/DATA /ds_job_salary/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    url = 'http://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword='+job_name+'&sc.keyword='+job_name+'&locT=C&locId=1132348&jobType='
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name('modal_closeIcon').click() #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="css-1uyte9r css-hca4ks e1wijj242"]').text
            except NoSuchElementException:
                continue 
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 

            #Printing for debugging
            if debugging:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if debugging:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [33]:
#This line will open a new chrome window and start the scraping.
#df = get_jobs("data analyst", 10, False)
#df

111111
Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Analyst,$42K-$81K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ",-1,201 to 500 Employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,-1
1,"Advertising Data Analyst, Business Intelligence",$52K-$99K (Glassdoor est.),Curious about what it's like to work at the wo...,3.6,Taboola\n3.6,"New York, NY",-1,1001 to 5000 Employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1
2,"Senior Data Analyst, Customer Operations",$85K-$146K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY",-1,1001 to 5000 Employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,-1
3,Senior Data Analyst,$81K-$142K (Glassdoor est.),About the Company:\nIndustrious is the largest...,3.6,Industrious\n3.6,"New York, NY",-1,201 to 500 Employees,2013,Company - Private,Membership Organizations,Business Services,Unknown / Non-Applicable,-1
4,Data Analyst,$39K-$71K (Glassdoor est.),Reproductive Medicine Associates of New York (...,3.1,RMA of NY\n3.1,"New York, NY",-1,51 to 200 Employees,-1,Private Practice / Firm,Health Care Services & Hospitals,Health Care,$10 to $25 million (USD),-1
5,Data Analyst,$46K-$65K (Glassdoor est.),"For more than a decade, Asembia has been worki...",3.5,Asembia\n3.5,"Florham Park, NJ",-1,501 to 1000 Employees,2004,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$5 to $10 million (USD),-1
6,Data Analyst - Predictive Modeler,$41K-$81K (Glassdoor est.),Zeta Global is looking for a talented Predicti...,3.3,Zeta Global\n3.3,"New York, NY",-1,1001 to 5000 Employees,2007,Company - Private,Advertising & Marketing,Business Services,$100 to $500 million (USD),-1
7,Senior Data Analyst,$60K-$110K (Glassdoor est.),About Known\n\nKnown is a modern marketing com...,3.5,Known\n3.5,"New York, NY",-1,201 to 500 Employees,2004,Company - Private,Advertising & Marketing,Business Services,$100 to $500 million (USD),-1
8,Salesforce Data Analyst,$60K-$103K (Glassdoor est.),"A pioneer in K–12 education since 2000, Amplif...",3.7,Amplify\n3.7,"Brooklyn, NY",-1,201 to 500 Employees,2000,Company - Private,K-12 Education,Education,$100 to $500 million (USD),-1
9,Data Analyst,$41K-$78K (Glassdoor est.),"Sony Electronics, Inc. look for the risk-taker...",4.2,Sony Electronics\n4.2,"Paramus, NJ",-1,10000+ Employees,1946,Subsidiary or Business Segment,Consumer Products Manufacturing,Manufacturing,$2 to $5 billion (USD),-1


In [34]:
df.head()


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Analyst,$42K-$81K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ",-1,201 to 500 Employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,-1
1,"Advertising Data Analyst, Business Intelligence",$52K-$99K (Glassdoor est.),Curious about what it's like to work at the wo...,3.6,Taboola\n3.6,"New York, NY",-1,1001 to 5000 Employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1
2,"Senior Data Analyst, Customer Operations",$85K-$146K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY",-1,1001 to 5000 Employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,-1
3,Senior Data Analyst,$81K-$142K (Glassdoor est.),About the Company:\nIndustrious is the largest...,3.6,Industrious\n3.6,"New York, NY",-1,201 to 500 Employees,2013,Company - Private,Membership Organizations,Business Services,Unknown / Non-Applicable,-1
4,Data Analyst,$39K-$71K (Glassdoor est.),Reproductive Medicine Associates of New York (...,3.1,RMA of NY\n3.1,"New York, NY",-1,51 to 200 Employees,-1,Private Practice / Firm,Health Care Services & Hospitals,Health Care,$10 to $25 million (USD),-1
